**Extract the data from csv**

In [10]:
import numpy as np

# We will use the StandardScaler module, so we can later deploy the model
from sklearn.preprocessing import StandardScaler

import pickle

# Load the data
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')

# The inputs are all columns in the csv, except for the first one and the last one
# The first column is the arbitrary ID, while the last contains the targets

unscaled_inputs_all = raw_csv_data[:,1:-1]

# The targets are in the last column. That's how datasets are conventionally organized.
targets_all = raw_csv_data[:,-1]

**Balance the dataset**

In [11]:
# There are different Python packages that could be used for balancing
# Here we approach the problem manually, so you can observe the inner workings of the balancing process

# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
# Declare a variable that will do that:
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

**Standardize the input**

In [12]:
# Crete a standar scaler object
scaler_deep_learning = StandardScaler()
# Fit and transform the original data
# Essentially, we calculate and STORE the mean and variance of the data in the scaler object
# At the same time we standrdize the data using this information
# Note that the mean and variance remain recorded in the scaler object
scaled_inputs = scaler_deep_learning.fit_transform(unscaled_inputs_equal_priors)

**Shuffle the data**

In [13]:
# When the data was collected it was actually arranged by date
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

# split the dataset into train validation and test

In [14]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. Note that each time you rerun this code, 
# you will get different values, as each time they are shuffled randomly.
# Normally you preprocess ONCE, so you need not rerun this code once it is done.
# If you rerun this whole sheet, the npzs will be overwritten with your newly preprocessed data.

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)


1777.0 3579 0.4965074043028779
225.0 447 0.5033557046979866
235.0 448 0.5245535714285714


# save three data set in .npz file

In [15]:
# Save the three datasets in *.npz.
# In the next lesson, you will see that it is extremely valuable to name them in such a coherent way!

np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

# save the scaler

In [16]:
# Similar to how we have saved the scaler files before, we also save this scaler, so we can apply in on new data
pickle.dump(scaler_deep_learning, open('scaler_deep_learning.pickle', 'wb'))

# **MACHINE LEARNING ALGORITHM**

**Import the library**

In [17]:
import numpy as np
import tensorflow as tf

**Data**

In [18]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs,validation_targets = npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

**Model**

In [21]:
input_size = 10
output_size = 2
hidden_layers_size = 50

model = tf.keras.Sequential([tf.keras.layers.Dense(hidden_layers_size,activation= 'relu'),
                             tf.keras.layers.Dense(hidden_layers_size,activation= 'relu'),
                             tf.keras.layers.Dense(output_size,activation='softmax'),
                             ])
model.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy',metrics = ['accuracy'])
batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(
    train_inputs,train_targets,
    batch_size = batch_size,
    epochs = max_epochs,
    callbacks = [early_stopping],
    validation_data = (validation_inputs,validation_targets),
    verbose = 2
)

Epoch 1/100
36/36 - 0s - loss: 0.5737 - accuracy: 0.7619 - val_loss: 0.4744 - val_accuracy: 0.8210
Epoch 2/100
36/36 - 0s - loss: 0.3876 - accuracy: 0.8723 - val_loss: 0.3951 - val_accuracy: 0.8501
Epoch 3/100
36/36 - 0s - loss: 0.3239 - accuracy: 0.8857 - val_loss: 0.3727 - val_accuracy: 0.8523
Epoch 4/100
36/36 - 0s - loss: 0.3002 - accuracy: 0.8902 - val_loss: 0.3443 - val_accuracy: 0.8546
Epoch 5/100
36/36 - 0s - loss: 0.2852 - accuracy: 0.8983 - val_loss: 0.3284 - val_accuracy: 0.8658
Epoch 6/100
36/36 - 0s - loss: 0.2760 - accuracy: 0.8975 - val_loss: 0.3303 - val_accuracy: 0.8591
Epoch 7/100
36/36 - 0s - loss: 0.2677 - accuracy: 0.9008 - val_loss: 0.3192 - val_accuracy: 0.8658
Epoch 8/100
36/36 - 0s - loss: 0.2614 - accuracy: 0.9061 - val_loss: 0.3114 - val_accuracy: 0.8658
Epoch 9/100
36/36 - 0s - loss: 0.2580 - accuracy: 0.9053 - val_loss: 0.3208 - val_accuracy: 0.8613
Epoch 10/100
36/36 - 0s - loss: 0.2527 - accuracy: 0.9067 - val_loss: 0.3104 - val_accuracy: 0.8702
Epoch 11/

**Test dataset validation**

In [22]:
test_loss , test_accurcy = model.evaluate(test_inputs,test_targets)  

14/14 [==============================] - 0s 3ms/step - loss: 0.2114 - accuracy: 0.9219


**Obtain the probability for a customer to convert**

In [24]:
model.predict(test_inputs).round(2)

array([[0.9 , 0.1 ],
       [0.9 , 0.1 ],
       [0.  , 1.  ],
       [0.1 , 0.9 ],
       [1.  , 0.  ],
       [0.  , 1.  ],
       [0.98, 0.02],
       [0.  , 1.  ],
       [0.92, 0.08],
       [0.94, 0.06],
       [0.  , 1.  ],
       [0.12, 0.88],
       [1.  , 0.  ],
       [0.82, 0.18],
       [0.87, 0.13],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.5 , 0.5 ],
       [0.56, 0.44],
       [0.01, 0.99],
       [1.  , 0.  ],
       [0.22, 0.78],
       [0.85, 0.15],
       [0.65, 0.35],
       [0.  , 1.  ],
       [0.8 , 0.2 ],
       [0.07, 0.93],
       [1.  , 0.  ],
       [0.14, 0.86],
       [0.07, 0.93],
       [0.11, 0.89],
       [1.  , 0.  ],
       [0.86, 0.14],
       [0.  , 1.  ],
       [0.93, 0.07],
       [0.  , 1.  ],
       [0.84, 0.16],
       [0.77, 0.23],
       [0.  , 1.  ],
       [1.  , 0.  ],
       [0.07, 0.93],
       [0.1 , 0.9 ],
       [0.93, 0.07],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.12, 0.88],
       [1.  ,

In [26]:
model.predict(test_inputs)[:,1].round(2)

array([0.1 , 0.1 , 1.  , 0.9 , 0.  , 1.  , 0.02, 1.  , 0.08, 0.06, 1.  ,
       0.88, 0.  , 0.18, 0.13, 1.  , 1.  , 0.5 , 0.44, 0.99, 0.  , 0.78,
       0.15, 0.35, 1.  , 0.2 , 0.93, 0.  , 0.86, 0.93, 0.89, 0.  , 0.14,
       1.  , 0.07, 1.  , 0.16, 0.23, 1.  , 0.  , 0.93, 0.9 , 0.07, 1.  ,
       1.  , 1.  , 0.88, 0.  , 0.07, 0.31, 1.  , 0.12, 0.01, 0.  , 0.  ,
       1.  , 0.21, 0.  , 0.97, 0.86, 0.06, 0.06, 0.  , 0.02, 0.03, 0.9 ,
       0.68, 0.42, 0.  , 1.  , 0.18, 1.  , 0.07, 1.  , 0.91, 0.  , 0.21,
       0.83, 1.  , 0.88, 0.  , 0.  , 0.14, 0.93, 1.  , 0.13, 1.  , 0.83,
       0.1 , 0.89, 0.92, 1.  , 0.89, 0.  , 0.99, 0.89, 0.8 , 0.8 , 0.  ,
       0.  , 1.  , 0.34, 0.91, 0.99, 0.93, 1.  , 0.04, 0.  , 1.  , 0.  ,
       0.11, 0.  , 0.86, 1.  , 1.  , 0.  , 0.74, 0.8 , 1.  , 1.  , 1.  ,
       0.84, 0.12, 0.88, 0.86, 0.  , 0.87, 0.  , 0.08, 0.86, 0.8 , 0.24,
       0.  , 0.04, 0.08, 1.  , 0.88, 1.  , 0.91, 0.  , 0.87, 0.12, 0.98,
       0.  , 0.99, 1.  , 0.07, 0.07, 0.93, 0.  , 0.

In [27]:
#for multiclass problem
np.argmax(model.predict(test_inputs),axis =1)

array([0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,

save the model

In [28]:
model.save('audiobook_model.h5')